<center>  <h1> Lab 4: Batch Normalization and Text CNN's </h1> </center> 
<center>  <h1> SOLUTIONS </h1> </center> 
<center> Jillian Fisher, Zaid Harchaoui </center>
    <center> Data 598 (Winter 2023), University of Washington </center>
    <center>  </center>

In this lab, we will study batch normalization and CNN's for text. 
For simplicity, we will stick to multilayer perceptrons (MLPs) for batch normalization
but the same ideas extend directly to convolutional neural networks.


This notebook is inspired by the [D2L book](https://d2l.ai/), http://debajyotidatta.github.io/nlp/deep/learning/word-embeddings/2016/11/27/Understanding-Convolutions-In-Text/, https://tzuruey.medium.com/using-convolution-neural-networks-to-classify-text-in-pytorch-3b626a42c3ca,  and adapted from lecture material created by Krishna Pillutla.




## Standardization in linear regression: An optimization viewpoint

Recall that we always normalize our features to be zero mean and unit variance 
(or something similar) on our training set.

Let us stack our input data $x_1, \cdots, x_n \in \mathbb{R}^d$ into a matrix $X \in \mathbb{R}^{n \times d}$. Denote the coordinate-wise mean and standard deviation as 
$$
    \mu^{(j)} = \frac{1}{n} \sum_{i=1}^n x_i^{(j)} \,, \quad
    \sigma^{(j)} = \sqrt{\frac{1}{n} (x_i^{(j)} - \mu^{(j)})^2} \,,
$$
for $j = 1, \cdots, d$. 
We can simply write the standardization as
$$
    \hat x_i = \frac{1}{\sigma}(x_i - \mu)
$$
where the operations are applied elementwise. 
We have applied this transformation to our data in each lab so far. 


Consider the linear prediction model 
$
f(x) = w^\top x + b
$
to predict outputs $y \approx f(x)$ from inputs $x$. Given $n$ input-output samples $(x_1, y_1), \ldots, (x_n, y_n) \in \mathbb{R}^d \times \mathbb{R}$, minimizing the least square loss over the training data reads
$$
\min_{w,b} \frac{1}{n}\sum_{i=1}^n  (y_i - w^\top x_i - b)^2.
$$

**Mean Normalization (Homework exercise)**:
Show that minimizing in $b$ amounts to center both inputs and outputs leading to 
$$
\min_{w} f(w) = \frac{1}{n}\sum_{i=1}^n  (\widetilde y_i - w^\top \hat x_i)^2.
$$
where $\widetilde y$ and $\hat x$ are respectively the centered outputs and inputs.


Recall now that from an optimization point of view, the step-size of a gradient step is controlled by the smoothness of the function, i.e. the Lipschitz-continuity of its gradient, which (for twice continuously differentiable functions) can simply computed as the largest eigenvalue of the Hessian matrix. 

**Variance Normalization**:
Similar to above, one can show that 
after normalizing the inputs, the objective  $f(w) = \frac{1}{n}\sum_{i=1}^n  (\widetilde y_i - w^\top \widehat x_i)^2$ has a smoothness constant no bigger than d.

In other words, feature normalization allows one to use a standard learning rate, invariant to the scale of the input. 

## Batch Normalization

We standardize our inputs in neural networks as well, but that only ensures that inputs to the first layer are normalized. What about inputs to subsequent layers?

**What is batch norm?**

Batch normalization is a technique which generalizes this standardization operation to ensure normalization of inputs in to an intermediate layer. 

In the linear model case, the mean and variance of our features do not change (since the data is fixed), therefore precomputing them once suffices. 
However, the inputs to hidden layers in neural networks changes over time. 
Batch norm, therefore, computes the mean and variance over a *minibatch of data*.

The batch norm layer gives the network two *trainable* parameters $\beta, \gamma$
which are learned over the course of training. If $\hat x_i$ is the normalized 
version of input $x_i$ to a batch norm layer, the output of the batch norm layer 
is 
$\gamma \hat x_i + \beta$.

**Note**: Batch norm requires large minibatch sizes such as 64 or 128 to work. The mean and standard deviation estimates computed from smaller batch sizes will be too noisy otherwise.

The pseudocode is given below. 

<img src="https://melfm.github.io/posts/2018-08-Understanding-Normalization/img/batch_norm_forward.png" width="500">


### Batch norm in action
Our first step is to notice that batch norm behaves differently at train and test times. 

At test time, there is no reason for a model to receive a large enough minibatch of inputs (think about the deep networks running on your smart phone).
Batch norm does not work if the minibatch size is too small. 

A solution to this issue is to estimate a running mean and standard deviation of the data encountered during training and use this as a proxy at test time. 

As a result, we need to tell PyTorch to switch from training to evaluation mode and vice-versa using `model.train()` and `model.eval()`, where `model` is a `torch.nn.Module`. 

**Part 1: Batch norm in action**:
Let us visualize how batch norm behaves differently at train and test times on a multilayer perceptron (MLP).


In [79]:
import numpy as np
import torch
from torchvision.datasets import FashionMNIST
from torch.nn.functional import cross_entropy, relu
import time
import copy

import matplotlib.pyplot as plt 
%matplotlib inline 

torch.manual_seed(0)
np.random.seed(1)

We use PyTorch's `BatchNorm1d` function. It is called so because the 
input a batch of vectors in $\mathbb{R}^d$ (in the form of a matrix of size $n\times d$). 

In [80]:
# pass in input dimensionality (it needs to create mean/std vectors of this dimension)
batch_norm_layer = torch.nn.BatchNorm1d(num_features=10)

# This is an instance of torch.nn.Module; we can use it as we would a module
isinstance(batch_norm_layer, torch.nn.Module)

True

In [81]:
# Let us create some synthetic data with a nonzero mean
mean_vector = 20 * torch.rand(10)  # non-zero mean
std_vector = 10 * torch.rand(10)

def get_random_data():
    return std_vector[None] * torch.randn(25, 10) + mean_vector[None]  # mean = mean_vector
random_data = get_random_data()

## Let us go to evaluation mode
batch_norm_layer.eval()

# print mean and variance of the data
print('Input mean:')
print(random_data.mean(dim=0))

print('Input std:')
print(random_data.std(dim=0))

Input mean:
tensor([ 9.3583, 13.6268,  1.7787,  2.8271,  7.5137, 13.0383, 11.1088, 19.5494,
         9.4111, 12.9476])
Input std:
tensor([3.6872, 3.6393, 0.2201, 1.5782, 2.5794, 5.5826, 6.1900, 5.1627, 1.4695,
        2.8457])


In [82]:
# Let us run our data through the batch norm layer and register its output
output = batch_norm_layer(random_data)

print('Output mean:')
print(output.mean(dim=0))

print('Output std:')
print(output.std(dim=0))

Output mean:
tensor([ 9.3582, 13.6268,  1.7787,  2.8271,  7.5137, 13.0382, 11.1087, 19.5493,
         9.4110, 12.9476], grad_fn=<MeanBackward1>)
Output std:
tensor([3.6872, 3.6393, 0.2201, 1.5782, 2.5794, 5.5826, 6.1900, 5.1627, 1.4695,
        2.8457], grad_fn=<StdBackward0>)


We do not observe any difference because batch norm in evaluation mode uses 
historical statistics of mean and std. But since this batch norm layer was just initialized, 
it has no history to go by.

Hence, it uses its defaults: $\mu = 0$ and $\sigma = 1$, which makes this batch norm layer just the identity map.

Let us switch to the train mode now. 


In [83]:
batch_norm_layer.train()  # switch to train mode

output = batch_norm_layer(random_data)

print('Output mean:')
print(output.mean(dim=0))

print('Output std:')
print(output.std(dim=0))

Output mean:
tensor([ 2.8610e-08, -7.6294e-08,  2.2888e-07, -3.8147e-08, -6.1989e-08,
        -1.5259e-07,  1.0967e-07,  1.1444e-07,  8.3923e-07, -2.7657e-07],
       grad_fn=<MeanBackward1>)
Output std:
tensor([1.0206, 1.0206, 1.0205, 1.0206, 1.0206, 1.0206, 1.0206, 1.0206, 1.0206,
        1.0206], grad_fn=<StdBackward0>)


And voilà! It works as expected. Let us run a few more random samples and investigate the evaluation mode further. 

In [84]:
for i in range(1000):  # so that batch norm updates its internal statistics
    random_data = get_random_data()
    output = batch_norm_layer(random_data)
    
batch_norm_layer.eval()  # switch to eval mode

random_data = get_random_data()
# print mean and variance of the data
print('Input mean:')
print(random_data.mean(dim=0))

print('Input std:')
print(random_data.std(dim=0))

output = batch_norm_layer(random_data)

print('Output mean:')
print(output.mean(dim=0))

print('Output std:')
print(output.std(dim=0))

Input mean:
tensor([ 9.9646, 16.0109,  1.8047,  3.2935,  6.6255, 13.3679,  8.3322, 19.1193,
         8.8255, 12.2329])
Input std:
tensor([3.8378, 3.5806, 0.2178, 1.6437, 2.4663, 5.0291, 6.6465, 7.5992, 1.4084,
        2.6209])
Output mean:
tensor([ 0.0882,  0.2729,  0.1629,  0.4320,  0.1669,  0.1631, -0.1770,  0.1296,
        -0.1818, -0.2027], grad_fn=<MeanBackward1>)
Output std:
tensor([1.0888, 0.9206, 1.0158, 0.9874, 0.8529, 0.9932, 0.9495, 0.9586, 0.8898,
        0.9480], grad_fn=<StdBackward0>)


Our data is much better normalized now!

## Part 2: Batch norm + MLPs
Just as in the convex case, batch norm helps with learning rates in the nonconvex case as well, especially when the networks are very deep. 

We start with a simple MLP module. Please read it carefully and understand what is going on. 

In [85]:
class MultiLayerPerceptron(torch.nn.Module):
    def __init__(self, input_dim, output_dim, 
                 num_hidden_layers, hidden_width, 
                 use_batch_norm=False):
        ## pass in how many hidden layers to use and width of each layer
        ## We will use the same width for all hidden layers
        super().__init__()  # call constructor of a super class
        self.num_hidden_layers = num_hidden_layers
        
        # Construct linear layers: use ModuleList to hold a list of submodules
        self.hidden_layers = torch.nn.ModuleList([torch.nn.Linear(input_dim, hidden_width)])
         # input -> hidden
        
        for i in range(num_hidden_layers-1):
            # hidden i -> hidden i+1
            self.hidden_layers.append(torch.nn.Linear(hidden_width, hidden_width))
            
        # hidden -> output
        self.hidden_to_output = torch.nn.Linear(hidden_width, output_dim)
            
        # construct batch norm layers
        self.use_batch_norm = use_batch_norm
        if use_batch_norm:
            print('using batch norm')
            self.batch_norm_layers = torch.nn.ModuleList(
                [torch.nn.BatchNorm1d(hidden_width)
                 for _ in range(num_hidden_layers)]
            )
        else:
            print('not using batch norm')
            
    def forward(self, x,):
        for i in range(self.num_hidden_layers):
            # apply self.hidden_layers[i] to x
            x = self.hidden_layers[i](x)
            # apply relu to x
            x = relu(x)
            # apply batch norm if required
            if self.use_batch_norm:
                x = self.batch_norm_layers[i](x)
        # hidden -> output
        return self.hidden_to_output(x)
            

In [86]:
model = MultiLayerPerceptron(input_dim=784, output_dim=10, hidden_width=64,  
                              num_hidden_layers=3, 
                              use_batch_norm=True)

print(model)

using batch norm
MultiLayerPerceptron(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
  )
  (hidden_to_output): Linear(in_features=64, out_features=10, bias=True)
  (batch_norm_layers): ModuleList(
    (0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


Let us load our FashionMNIST dataset again.

In [87]:
# download dataset (~117M in size)
train_dataset = FashionMNIST('./data', train=True, download=True)
X_train = train_dataset.data # torch tensor of type uint8
y_train = train_dataset.targets # torch tensor of type Long
test_dataset = FashionMNIST('./data', train=False, download=True)
X_test = test_dataset.data
y_test = test_dataset.targets

# choose a subsample of 10% of the data:
idxs_train = torch.from_numpy(
    np.random.choice(X_train.shape[0], replace=False, size=X_train.shape[0]//10))
X_train, y_train = X_train[idxs_train], y_train[idxs_train]
# idxs_test = torch.from_numpy(
#     np.random.choice(X_test.shape[0], replace=False, size=X_test.shape[0]//10))
# X_test, y_test = X_test[idxs_test], y_test[idxs_test]

print(f'X_train.shape = {X_train.shape}')
print(f'n_train: {X_train.shape[0]}, n_test: {X_test.shape[0]}')
print(f'Image size: {X_train.shape[1:]}')

# Normalize dataset: pixel values lie between 0 and 255
# Normalize them so the pixelwise mean is zero and standard deviation is 1

X_train = X_train.float()  # convert to float32
X_train = X_train.view(-1, 784)  # flatten into a (n, d) shape
mean, std = X_train.mean(axis=0), X_train.std(axis=0)
X_train = (X_train - mean[None, :]) / (std[None, :] + 1e-6)  # avoid divide by zero

X_test = X_test.float()
X_test = X_test.view(-1, 784)
X_test = (X_test - mean[None, :]) / (std[None, :] + 1e-6)

n_class = np.unique(y_train).shape[0]

X_train.shape = torch.Size([6000, 28, 28])
n_train: 6000, n_test: 10000
Image size: torch.Size([28, 28])


We now write the training utilities. 

**Note**: We must pay extra care to ensure that the model is in training or evaluation mode when appropriate.
Batch norm behaves differently at train time and test time.

In [88]:
def compute_objective(model, X, y):
    """ Compute the multinomial logistic loss. 
        model is a module
        X of shape (n, d) and y of shape (n,)
    """
    # send 
    score = model(X)
    # PyTorch's function cross_entropy computes the multinomial logistic loss
    return cross_entropy(input=score, target=y, reduction='mean') 

@torch.no_grad()
def compute_accuracy(model, X, y):
    """ Compute the classification accuracy
        ws is a list of tensors of consistent shapes 
        X of shape (n, d) and y of shape (n,)
    """
    is_train = model.training  # if True, model is in training mode
    model.eval()  # use eval mode for accuracy
    score = model(X)
    predictions = torch.argmax(score, axis=1)  # class with highest score is predicted
    if is_train:  # switch back to train mode if appropriate
        model.train()
    return (predictions == y).sum() * 1.0 / y.shape[0]

@torch.no_grad()
def compute_logs(model, verbose=False):
    is_train = model.training  # if True, model is in training mode
    model.eval()  # switch to eval mode
    train_loss = compute_objective(model, X_train, y_train)
    test_loss = compute_objective(model, X_test, y_test)
    train_accuracy = compute_accuracy(model, X_train, y_train)
    test_accuracy = compute_accuracy(model, X_test, y_test)
    if verbose:
        print(('Train Loss = {:.3f}, Train Accuracy = {:.3f}, ' + 
               'Test Loss = {:.3f}, Test Accuracy = {:.3f}').format(
                train_loss.item(), train_accuracy.item(), 
                test_loss.item(), test_accuracy.item())
    )
    if is_train:  # switch back to train mode if appropriate
        model.train()
    return (train_loss, train_accuracy, test_loss, test_accuracy)

Now we write the minibatch SGD function (copied over from a previous lab). 
Note again that we must pay special care to making sure the model is in training mode

In [89]:
def minibatch_sgd_one_pass(model, X, y, learning_rate, batch_size, verbose=False):
    model.train()
    num_examples = X.shape[0]
    average_loss = 0.0
    num_updates = int(round(num_examples / batch_size))
    for i in range(num_updates):
        idxs = np.random.choice(X.shape[0], size=(batch_size,)) # draw `batch_size` many samples
        model.train()  # make sure we are in train mode
        # compute the objective. 
        objective = compute_objective(model, X[idxs], y[idxs]) 
        average_loss = 0.99 * average_loss + 0.01 * objective.item()
        if verbose and (i+1) % 100 == 0:
            print(average_loss)
        
        # compute the gradient using automatic differentiation
        gradients = torch.autograd.grad(outputs=objective, inputs=model.parameters())
        
        # perform SGD update. IMPORTANT: Make the update inplace!
        with torch.no_grad():
            for (w, g) in zip(model.parameters(), gradients):
                w -= learning_rate * g
    return model

Our goal now is to find the divergent learning rates 
with and without batch norm. We use a fixed batch size of 32

Use a hidden width of 64 throughout. 
Vary the depth of the network
(i.e., the number of hidden layers)
between 1 and 8.

In [90]:
num_hidden_layers = 25

# no batch norm
model = MultiLayerPerceptron(input_dim=784, output_dim=10, hidden_width=64,  
                              num_hidden_layers=num_hidden_layers, 
                              use_batch_norm=False)


learning_rate = 0.01

logs = []

logs.append(compute_logs(model, verbose=True))

batch_size = 32

for _ in range(batch_size*2):  # run multiple passes because our sub-sampled dataset is too small
    model = minibatch_sgd_one_pass(model, X_train, y_train, learning_rate, 
                                   batch_size=batch_size, verbose=True)


not using batch norm
Train Loss = 2.305, Train Accuracy = 0.098, Test Loss = 2.306, Test Accuracy = 0.100
1.4610244231938492
1.459454987796917
1.4601824636020182
1.4601957181051841
1.4596348277282227
1.459758722001808
1.459724082906664
1.459888446296574
1.4605523654208508
1.4602426229716317
1.4596335457293244
1.4600472015893164
1.4596916614444821
1.4595547688885508
1.4595775951869876
1.459566127352911
1.459253654706653
1.459558534182161
1.459748457194877
1.459541548060624
1.458791563649635
1.459197666100485
1.4599110055244129
1.4600266489496583
1.4593013398284782
1.4600307546099458
1.4593770488230888
1.45895676442173
1.459398770227837
1.4596061542967302
1.459977881629981
1.4598174398982913
1.4596942845498477
1.4601149311301247
1.4594260978013025
1.4593885459730447
1.4591177991081752
1.4600294882359568
1.4593796413550661
1.4598271469078352
1.4590010752881084
1.4591937226424145
1.459816625884016
1.4602565087200026
1.4587503487837823
1.4588108040709156
1.459703557086519
1.4600616629423249

In [91]:
# with batch norm
model = MultiLayerPerceptron(input_dim=784, output_dim=10, hidden_width=64,  
                              num_hidden_layers=num_hidden_layers, 
                              use_batch_norm=True)


learning_rate = 1.28

logs = []

logs.append(compute_logs(model, verbose=True))

batch_size = 32

for _ in range(batch_size):  # run multiple passes because our sub-sampled dataset is too small
    model = minibatch_sgd_one_pass(model, X_train, y_train, learning_rate, 
                                   batch_size=batch_size, verbose=True)


using batch norm
Train Loss = 2.303, Train Accuracy = 0.100, Test Loss = 2.304, Test Accuracy = 0.100
1.3052448224245976
1.0041311674302094
1.1985350510943467
1.132449850165345
0.9940929182861032
0.9751480897977858
0.9326149993757314
0.8867976356882423
0.8971497463342714
0.8768544997353401
0.8782271612311081
0.8719234527397317
0.8210365360066042
0.8397398748234253
0.7882526543423359
0.7943221293121556
0.7916193347101231
0.74583143826128
0.7533117222659959
0.724431658797501
0.6911159795570911
0.744039916609071
0.6960081046017008
0.9596366051768203
0.8005560476055051
1.102533185476845
1.0789373949179668
1.0290348359022192
1.0185691574843514
0.9911090865461109
0.9558448767449422
0.9458174577526165


Redo the last part with depths of $10$ and $25$, and $50$. What do you observe?

**NOTE**: the larger models (depth > 20) get very slow. Use a cloud computing resource if it is too slow on your laptop.

**Hint**: If some of the models at these depths get stuck or suddenly diverge to infinity/NaN, know that it is an often encountered problem with very deep networks. A combination of many factors can overcome this issue: careful initialization and tuning of learning rates as well as dropout. The most popular fix, however, is to use residual networks, know as _ResNets_.

## Exploring CNN for Text
Now, we will explore convoluation networks more thoroughly using the text example from the demo. 

First, we download the data and tokenizer the input. We will again use the Spacy tokenization.

In [92]:
# Install docopt and spacy
%%capture
# If this give an error, in your terminal activate the data598 env "conda activate data598" and then install each
#package seperately: "pip install spacy", "conda install docopt"
import pip
!pip install spacy
import spacy

!python3 -m spacy download en_core_web_sm

UsageError: Line magic function `%%capture` not found.


In [93]:
# Download Text REtrietest Conference (TREC) Question Classifiction data (https://cogcomp.seas.upenn.edu/Data/QA/QC/)
import requests

# Training Data
URL = "https://cogcomp.seas.upenn.edu/Data/QA/QC/train_1000.label"
TREC_train_data = str(requests.get(URL).content).split("\\n")
train_label = []
train_input = []
for i, t in enumerate(TREC_train_data[:-1]):
    if i ==0:
        train_label.append(t.split(":")[0][2:])
    else:
      train_label.append(t.split(":")[0])
    train_input.append(" ".join(t.split(":")[1].split(" ")[1:]))

train={"input":train_input, "label":train_label}

# Test Data
URL = "https://cogcomp.seas.upenn.edu/Data/QA/QC/TREC_10.label"
TREC_test_data = str(requests.get(URL).content).split("\\n")
test_label = []
test_input = []
for i, t in enumerate(TREC_test_data[:-1]):
    if i ==0:
        test_label.append(t.split(":")[0][2:])
    else:
      test_label.append(t.split(":")[0])
    test_input.append(" ".join(t.split(":")[1].split(" ")[1:]))
test={"input":test_input, "label":test_label}

print("Example of Input/Label")
print("Input:", train_input[1])
print("Label:", train_label[1])

np.unique(list(train['label'])+list(test['label']))

Example of Input/Label
Input: What films featured the character Popeye Doyle ?
Label: ENTY


array(['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM'], dtype='<U4')

In [94]:
# Change string label to integer label
from sklearn import preprocessing
import numpy as np

le = preprocessing.LabelEncoder()
le.fit(train['label']+test['label'])
train['categorical_label'] = le.transform(train['label'])
test['categorical_label'] = le.transform(test['label'])
print("New Label:", train['categorical_label'][1])
print("Nclasses:", len(np.unique(list(train['categorical_label'])+list(test['categorical_label']))))

New Label: 2
Nclasses: 6


In [95]:
# Tokenize the data
import spacy
spacy.load('en_core_web_sm')
nlp = spacy.load('en_core_web_sm')

#  extract features training_data
train_input_vec = [[nlp(v).vector] for v in train['input']]  
test_input_vec = [[nlp(v).vector] for v in test['input']]

print("Length of One word Vector:", len(train_input_vec[0][0]))
print("Word Vector Example: ", train_input_vec[1])

Length of One word Vector: 96
Word Vector Example:  [array([ 0.05669461,  0.14431228, -0.09996024,  0.24405436,  0.17144223,
       -0.00698099,  0.5538776 , -0.04801345,  0.10472064, -0.0724301 ,
        0.46154702,  0.6193586 , -0.2742111 , -0.36640358, -0.41472915,
       -0.3367687 , -0.80017823,  0.01405566,  0.37896937, -0.26699197,
        0.06921677,  0.1596782 ,  0.4810588 , -0.59376097,  0.54818946,
        0.47652596,  0.55965155, -0.21928822,  0.08473992,  0.5288323 ,
       -0.47222984, -0.03617568, -0.03886646, -0.4117626 , -0.05733662,
        0.00477944, -0.20796035,  0.08200786,  0.00801046,  0.29752997,
       -0.22893679,  0.33473495,  0.16553667, -0.12274137, -0.3228015 ,
       -0.28731245,  0.04599106,  0.05031565,  0.18770975,  0.06129264,
       -0.1034981 , -0.06582367,  0.57545704,  0.15352717, -0.22472565,
       -0.04059392,  0.11420984,  0.40521163,  0.3088222 , -0.2678545 ,
       -0.24282809,  0.06950298, -0.37544304, -0.21284604,  0.14027752,
       -0.0

Now, let's explore how the accuracy changes as we change the kernel_size. 
Create a model and try changing the kernel_size and padding.

Kernel   Padding

2           1

3           1

4           2

5           2

For each pair run the model five times and average the accuracy.  How does mean accuracy change?
*Hint: you will have to change the linear dimentions to 1600, 1536, 1600, and 1536 correspondingly

In [96]:
from torch import nn
import torch
from torch import optim
import torch.nn.functional as F
class SimpleCNN(nn.Module):
    def __init__(self, nclasses:int, linear_dimension:int,kernel_size, padding, window_size: int = 16, embedding_dim: int = 16,
    filter_multiplier: int = 64):
        super(SimpleCNN, self).__init__()
        self.simpleconv = nn.Conv2d(in_channels=1, out_channels=filter_multiplier, stride=CONV_STRIDE, kernel_size=kernel_size, padding=padding)
        self.maxpool = nn.MaxPool2d(kernel_size=kernel_size, stride=CONV_STRIDE, padding=padding)
        self.linear = nn.Linear(linear_dimension, out_features=nclasses)
        self.softmax = nn.LogSoftmax(1)
    
    def forward(self, x, **kwargs): # shape: (batch, 16, 96)
        # input x is already embedding vectors from spaCy
        # set the channel dim to a new dimension that’s just 1
        x = torch.transpose(x, 1, 2) # (batch, 1, 96)
        x = torch.unsqueeze(x, 1) # (batch, 1, 96, 1)
        x = self.simpleconv(x) # (batch, 64, 48, 1)
        x = self.maxpool(x) # (batch, 64, 24, 1) 
        F.relu(x) # non-linear function to activate the neurons
        x = x.flatten(start_dim=1) # (batch, 1536)
        x = self.linear(x)
        F.relu(x)
        x = self.softmax(x)
        return x

In [97]:
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)

def run_cnn_text(num_classes, batch_size, learning_rate, num_epochs, CONV_STRIDE, linear_dimention, kernel_size, padding):
  model = SimpleCNN(num_classes, linear_dimention, kernel_size, padding) 
  # Create Dataloaders
  train_inputs_tensor, test_inputs_tensor, train_label_tensor, test_label_tensor = tuple(torch.tensor(data) for data in[train_input_vec, test_input_vec, train['categorical_label'], test['categorical_label']])
  # Create DataLoader for training data
  train_data = TensorDataset(train_inputs_tensor, train_label_tensor)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

  # Create DataLoader for testidation data
  test_data = TensorDataset(test_inputs_tensor, test_label_tensor)
  test_sampler = SequentialSampler(test_data)
  test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


  optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.8)
  lossfn = nn.NLLLoss()
  for epoch in range(num_epochs):
    running_loss = 0.
    for i, batch in enumerate(train_dataloader): # create a new generator every epoch
      input, label  = b_input_ids, b_labels = tuple(t for t in batch)
      optimizer.zero_grad()
      outputs = model(input)
      loss = lossfn(outputs, label)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()


  model = model.eval()
  num_correct = 0
  num_examples = 0
  # Check accuracy
  for i, batch in enumerate(test_dataloader):
    y, target = batch
    output = model(y)
    y_pred = []
    for o in output:
      y_pred.append(torch.max(o, -1).indices.item()) # Choose class with highest option
    y_pred = torch.FloatTensor(y_pred)
    correct = torch.eq(torch.round(y_pred).type(target.type()), target).view(-1)
    num_correct += torch.sum(correct).item()
    num_examples += correct.shape[0]
  print("Accuracy: ",  num_correct / num_examples)
  return(num_correct / num_examples)

In [98]:
# Train Model
num_classes = 6 
batch_size = 32
learning_rate = 0.1
num_epochs = 50
CONV_STRIDE = 2
linear_dimension = 1600
kernel_size = 2
padding = 1

test_accuracy = []
for i in range(5):
    test_accuracy.append(run_cnn_text(num_classes, batch_size, learning_rate, num_epochs, CONV_STRIDE, linear_dimension, kernel_size, padding))
print("Kernel = 2:", np.mean(test_accuracy))

linear_dimension = 1536
kernel_size = 3
padding = 1
test_accuracy = []
for i in range(5):
    test_accuracy.append(run_cnn_text(num_classes, batch_size, learning_rate, num_epochs, CONV_STRIDE, linear_dimension, kernel_size, padding))
print("Kernel = 3:", np.mean(test_accuracy))

linear_dimension = 1600
kernel_size = 4
padding = 2
test_accuracy = []
for i in range(5):
    test_accuracy.append(run_cnn_text(num_classes, batch_size, learning_rate, num_epochs, CONV_STRIDE, linear_dimension, kernel_size, padding))
print("Kernel = 4:", np.mean(test_accuracy))

linear_dimension = 1536
kernel_size = 5
padding = 2
test_accuracy = []
for i in range(5):
    test_accuracy.append(run_cnn_text(num_classes, batch_size, learning_rate, num_epochs, CONV_STRIDE, linear_dimension, kernel_size, padding))
print("Kernel = 5:", np.mean(test_accuracy))


Accuracy:  0.684
Accuracy:  0.66
Accuracy:  0.664
Accuracy:  0.652
Accuracy:  0.668
Kernel = 2: 0.6656000000000001
Accuracy:  0.662
Accuracy:  0.658
Accuracy:  0.674
Accuracy:  0.654
Accuracy:  0.654
Kernel = 3: 0.6604
Accuracy:  0.694
Accuracy:  0.68
Accuracy:  0.698
Accuracy:  0.686
Accuracy:  0.678
Kernel = 4: 0.6872
Accuracy:  0.702
Accuracy:  0.698
Accuracy:  0.694
Accuracy:  0.702
Accuracy:  0.672
Kernel = 5: 0.6936
